In [ ]:
from PIL import Image, ImageOps
import numpy as np
import copy
from scipy.sparse import csr_matrix
import re

i1 = Image.open('cleared_cube.png')
ancho, alto = i1.size
print(f'{ancho}, {alto}')

657, 487


In [ ]:
class cube:
    """
    Clase cubo que contiene 6 objetos _cube_face que corresponden a las caras del cubo.
    """
    class _cube_face:
        """
        Clase representativa de la cara de un cubo que contiene 9 celdas.
        """

        def __init__(self):
            """
            Constructor de la cara, con 1 array bidimensional 3x3 y 4 atributos usados para referenciar las caras vecinas.

            Attributes:
                matrix (numpy.ndarray)
                up (Nonetype): Vecino encima.
                down (Nonetype): Vecino debajo.
                left (Nonetype): Vecino a la izquierda.
                right (Nonetype): Vecino a la derecha.
            """
            self.matrix = np.zeros((3, 3))
            self.up = None
            self.down = None
            self.left = None
            self.right = None

        def links(self, up, down, left, right):
            """
            Función que enlaza las caras vecinas de una respectiva cara.

            Args:
                up (_cube_face): Vecino encima.
                down (_cube_face): Vecino debajo.
                left (_cube_face): Vecino a la izquierda.
                right (_cube_face): Vecino a la derecha.

            Attributes:
                up (_cube_face): Vecino encima.
                down (_cube_face): Vecino debajo.
                left (_cube_face): Vecino a la izquierda.
                right (_cube_face): Vecino a la derecha.
            """
            self.up = up
            self.down = down
            self.left = left
            self.right = right

        def clockwise(self):
            """
            Rotación a favor del reloj a partir de una cara.
            """
            # Rotación de la matriz de la cara.
            self.matrix = np.rot90(self.matrix, axes=(1, 0))

            # Rotación de las celdas afectadas de los vecinos.
            temp = copy.copy(self.right.matrix[:,0][::-1]) # copy.copy es para evitar se asigne una referencia. El valor es invertido.
            self.right.matrix[:,[0]] = np.array([self.up.matrix[2]]).T
            self.up.matrix[2] = self.left.matrix[:,2][::-1] # El valor es invertido.
            self.left.matrix[:,[2]] = np.array([self.down.matrix[0]]).T
            self.down.matrix[0] = temp

        def counterclockwise(self):
            """Rotación contrarreloj a partir de una cara"""

            # Rotación de la matriz de la cara.
            self.matrix = np.rot90(self.matrix, axes=(0, 1))

            # Rotación de las celdas afectadas de los vecinos.
            temp = copy.copy(self.left.matrix[:,2])
            self.left.matrix[:,[2]] = np.array([self.up.matrix[2][::-1]]).T
            self.up.matrix[2] = self.right.matrix[:,0]
            self.right.matrix[:,[0]] = np.array([self.down.matrix[0][::-1]]).T
            self.down.matrix[0] = temp

    def __init__(self):
        """
        Constructor del cubo, genera las 6 caras y las enlaza con sus respectivas vecinas.

        Attributes:
            F (_cube_face): Cara frontal (front).
            R (_cube_face): Cara derecha (right).
            B (_cube_face): Cara trasera (back).
            L (_cube_face): Cara izquierda (left).
            U (_cube_face): Cara superior (up).
            D (_cube_face): Cara inferior (down).
        """
        self.F = self._cube_face()
        self.R = self._cube_face()
        self.B = self._cube_face()
        self.L = self._cube_face()
        self.U = self._cube_face()
        self.D = self._cube_face()
        
        self.F.links(self.U, self.D, self.L, self.R)
        self.R.links(self.U, self.D, self.F, self.B)
        self.B.links(self.U, self.D, self.R, self.L)
        self.L.links(self.U, self.D, self.B, self.F)
        self.U.links(self.B, self.F, self.R, self.L)
        self.D.links(self.F, self.B, self.R, self.L)

    def show(self):
        """
        Muestra en consola el estado actual del cubo.
        """
        # Las caras unidas horizontalmente.
        middle_line = (self.L.matrix, self.F.matrix, self.R.matrix, self.B.matrix)
        # Las caras concatenadas en una sola matriz.
        middle = np.concatenate(middle_line, axis=1)

        # Formateando el texto.
        text = '\t  ' + f' {self.U.matrix}'.replace('\n', '\n\t  ')
        text += '\n\t   ' + '-------\n'
        for row in middle:
            for i in range(len(row)):
                text += f'{np.format_float_positional(row[i], precision=0)} '
                if ((i+1)%3 == 0 and (i+1 != len(row))):
                    text += f'| '
            text += '\n'
        text += '\t   -------'
        text += '\n\t  ' + f' {self.D.matrix}'.replace('\n', '\n\t  ')
        text = re.sub('\[|\]', '', text)
        print(text)

In [ ]:
a = cube()
a.F.matrix = np.array([[j+1+3*i for j in range(len(a.U.matrix))] for i in range(len(a.U.matrix))], dtype = np.float32)
a.U.matrix = np.array([[j+1+3*i for j in range(len(a.U.matrix))] for i in range(len(a.U.matrix))], dtype = np.float32)


a.show()
a.F.counterclockwise()
print('_'*50)
a.show()
a.F.counterclockwise()
print('_'*50)
a.show()
a.F.counterclockwise()
print('_'*50)
a.show()
a.F.counterclockwise()
print('_'*50)


In [ ]:
def calculation(matrix,center,opposite,startrow,startcolumn):
  sumdistance=0
  for i in range(startrow,startrow+3):
    for j in range(startcolumn,startcolumn+3):
      if(matrix[i][j]==opposite):
        sumdistance=sumdistance+2
      else:
         if(matrix[i][j]!=center):
           sumdistance=sumdistance+1
  print('sum of distance in this face is:')
  print(sumdistance)
  return sumdistance

In [ ]:
Distance=np.zeros((6,9))
TotalDistance=0
Distance[0:3,0:3]=[[1,2,3],[4,5,1],[5,3,4]]
Distance[0:3,3:6]=[[6,5,2],[5,4,2],[3,2,4]]
Distance[0:3,6:9]=[[1,1,3],[3,1,6],[1,5,3]]
Distance[3:6,0:3]=[[2,1,2],[5,3,2],[2,4,6]]
Distance[3:6,3:6]=[[5,6,4],[6,2,3],[1,3,6]]
Distance[3:6,6:9]=[[5,4,6],[1,6,4],[5,6,4]]
TotalDistance=TotalDistance+calculation(Distance,5,1,0,0)
TotalDistance=TotalDistance+calculation(Distance,4,6,0,3)
TotalDistance=TotalDistance+calculation(Distance,1,5,0,6)
TotalDistance=TotalDistance+calculation(Distance,3,2,3,0)
TotalDistance=TotalDistance+calculation(Distance,2,3,3,3)
TotalDistance=TotalDistance+calculation(Distance,6,4,3,6)
print('The total distance on this move is:')
print(TotalDistance)